## Classification

In [1]:
import io
import pandas as pd
import numpy as np
import tensorflow as tf
import random
from sklearn.preprocessing import normalize

In [2]:
# 데이터 - 자연유산 및 유도유산(낙태) 후 불임 여부

infert = pd.read_csv('data/infert.csv')
infert.head()

# education : 교육받은 년수. 0 = 0-5, 1 = 6-11, 2 = 12+ yrs.
# parity : 총 유산 횟수
# induced : 유도유산 횟수 (2 = 2 or more)
# spontaneous : 자연유산 횟수 (2 = 2 or more)
# stratum : 사회 계층
# case : 1 = 불임

,education,age,parity,induced,case,spontaneous,stratum,pooled_stratum
0,0,26,6,1,1,2,1,3
1,0,42,1,1,1,0,2,1
2,0,39,6,2,1,0,3,4
3,0,34,4,2,1,0,4,2
4,1,35,3,1,1,1,5,32


In [3]:
X_data = infert[["parity", "induced", "spontaneous", "stratum", "pooled_stratum"]]
y_data = infert["case"]

X_data = pd.DataFrame(normalize(X_data))

In [4]:
# train / test 분리
tf.set_random_seed(123)

train_idx = random.sample(list(range(len(infert))), int(len(infert) * 0.8 ))

train_X = X_data.iloc[train_idx]
train_y = y_data[train_idx]

test_X = X_data.drop(train_idx)
test_y = y_data.drop(train_idx)

print(train_X.shape, test_X.shape)

(198, 5) (50, 5)


In [5]:
# 네트워크 생성

X = tf.placeholder(shape = (None, 5), dtype = tf.float32)
y = tf.placeholder(shape = (None), dtype = tf.float32)

W1 = tf.Variable(initial_value= tf.random_normal([5, 1]), dtype = tf.float32)
b1 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)
L1 = tf.nn.relu(tf.add(tf.matmul(X, W1), b1))

W2 = tf.Variable(initial_value= tf.random_normal([1, 1]), dtype = tf.float32)
b2 = tf.Variable(initial_value= tf.random_normal([1]), dtype = tf.float32)
hypo = tf.add(tf.matmul(L1, W2), b2)

cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = y, logits = hypo))

optimizer = tf.train.GradientDescentOptimizer(learning_rate= 0.0001)

train = optimizer.minimize(cost)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
for i in range(1000):
    _, cost_val = sess.run([train, cost], feed_dict = {X: train_X, y: train_y})
    if i % 100 ==0:
        print("step: ", i, "\t", "cost: ", cost_val)
        
print("train finished!!")

step:  0 	 cost:  338.4491
step:  100 	 cost:  338.4491
step:  200 	 cost:  338.4491
step:  300 	 cost:  338.4491
step:  400 	 cost:  338.4491
step:  500 	 cost:  338.4491
step:  600 	 cost:  338.4491
step:  700 	 cost:  338.4491
step:  800 	 cost:  338.4491
step:  900 	 cost:  338.4491
train finished!!


In [8]:
predict = tf.cast(hypo > 0.5, dtype = tf.float32)

accuracy = tf.reduce_mean(tf.cast(tf.equal(predict, y), dtype = tf.float32 ))

print("accuracy: ", sess.run(accuracy, feed_dict = {X: test_X, y: test_y}))

accuracy:  0.38
